# Crossmatch with ZTF and PanSTARRS

Crossmatch the ComCam data with ZTF and PanSTARRS and store the results as HATS catalogs on disk.

In [ ]:
import os
import lsdb
import tempfile

from upath import UPath
from dask.distributed import Client
from nested_pandas import NestedDtype

In [ ]:
DRP_VERSION = os.environ["DRP_VERSION"]
print(f"DRP_VERSION: {DRP_VERSION}")
base_output_dir = UPath(f"/sdf/data/rubin/shared/lsdb_commissioning")
hats_dir = base_output_dir / "hats" / DRP_VERSION

In [ ]:
tmp_path = tempfile.TemporaryDirectory()
tmp_dir = tmp_path.name
client = Client(n_workers=16, threads_per_worker=1, local_directory=tmp_dir)

Let's load ZTF DR22 with lightcurves and PanSTARRS:

In [ ]:
ztf_dr22_lc = lsdb.read_hats(
    "https://data.lsdb.io/hats/ztf_dr22/ztf_lc",
    margin_cache="https://data.lsdb.io/hats/ztf_dr22/ztf_lc_10arcs",
)
ztf_dr22_lc

In [ ]:
panstarrs = lsdb.read_hats(
    UPath("s3://stpubdata/panstarrs/ps1/public/hats/otmo", anon=True),
    margin_cache=UPath(
        "s3://stpubdata/panstarrs/ps1/public/hats/otmo_10arcs", anon=True
    ),
)
panstarrs

And define a crossmatching radius:

In [ ]:
xmatch_radius_arcsec = 1

### Crossmatch DIA objects

In [ ]:
diaObject_lc = lsdb.read_hats(hats_dir / "diaObject_lc").map_partitions(
    lambda df: df.assign(
        **{
            lc_column: df[lc_column].astype(
                NestedDtype.from_pandas_arrow_dtype(df.dtypes[lc_column])
            )
            for lc_column in ["diaSource", "diaForcedSource"]
        }
    )
)
diaObject_lc

In [ ]:
diaObject_lc_x_ztf_dr22 = diaObject_lc.crossmatch(
    ztf_dr22_lc, radius_arcsec=xmatch_radius_arcsec, suffixes=("", "_ztf")
)
diaObject_lc_x_ztf_dr22.to_hats(hats_dir / "diaObject_lc_x_ztf_dr22")

In [ ]:
diaObject_lc_x_ps1 = diaObject_lc.crossmatch(
    panstarrs, radius_arcsec=xmatch_radius_arcsec, suffixes=("", "_ps1")
)
diaObject_lc_x_ps1.to_hats(hats_dir / "diaObject_lc_x_ps1")

### Crossmatch objects

In [ ]:
object_lc = lsdb.read_hats(hats_dir / "object_lc").map_partitions(
    lambda df: df.assign(
        **{
            "forcedSource": df["forcedSource"].astype(
                NestedDtype.from_pandas_arrow_dtype(df.dtypes["forcedSource"])
            )
        }
    )
)
object_lc

In [ ]:
object_lc_x_ztf_dr22 = object_lc.crossmatch(
    ztf_dr22_lc, radius_arcsec=xmatch_radius_arcsec, suffixes=("", "_ztf")
)
object_lc_x_ztf_dr22.to_hats(hats_dir / "object_lc_x_ztf_dr22")

In [ ]:
object_lc_x_ps1 = object_lc.crossmatch(
    panstarrs, radius_arcsec=xmatch_radius_arcsec, suffixes=("", "_ps1")
)
object_lc_x_ps1.to_hats(hats_dir / "object_lc_x_ps1")

In [ ]:
client.close()
tmp_path.cleanup()